In [ ]:
# imports
import requests
from splinter import Browser as browse
from bs4 import BeautifulSoup as bSoup
import pandas as pd
import time
import pymongo
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
con = 'mongodb://localhost:27017'
cli = pymongo.MongoClient(con)
cli.drop_database('mars_db')

In [ ]:
db = cli.mars_db
collection = db.articles

In [ ]:
executable_path = {'executable_path' : ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)

In [ ]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
mars = {}

results = soup.find_all('div', class_='content_title')
title = results[1].a.text

results = soup.find_all('div', class_= 'article_teaser_body')
para = results[0].text

mars['news_title'] = title
mars['news_paragraph'] = para

In [ ]:
# JPL Mars Space Image
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'

browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = browser.find_by_tag('button')[1]
results.click()
img_results = soup.find_all('img')[1]['src']
mars['featured_image_url'] = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + img_results

In [ ]:
# Mars Facts
url = 'https://space-facts.com/mars/'

table = pd.read_html(url)[0]
table.columns=['Description', 'Mars']
table.set_index('Description', inplace = True)
mars_facts = table.to_html('templates/Mars_fact_table.html')

In [ ]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('div', class_ = 'description')

img_urls = []

for name in results:
    temp = name.find('h3').text
    two_words = temp[:-9]

    browser.click_link_by_partial_text(temp)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find_all('img')[5]['src']
    base_url = 'astrogeology.usgs.gov'
    img_url = base_url + img
    dic_pic = {'title':two_words,'img_url':img_url}
    
    img_urls.append(dic_pic)
    
    browser.back()
    
for i in len(img_url):
    mars['title'].append(img_url[i].title)
    mars['img_url'].append(img_url[i].img_url)

In [ ]:
browser.quit()
collection.insert_one(mars)